In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

os.environ["PATH_TO_ENV"] = "~/projects/chatsky-llm-autoconfig/.env"

In [ ]:
from three_stages_llm import ThreeStagesGraphGenerator as LLMGenerator
from three_stages_algo import ThreeStagesGraphGenerator as AlgoGenerator
from three_stages_extender import ThreeStagesGraphGenerator as LLMExtender

In [ ]:
from dialogue2graph.pipelines.core.graph import Graph
from dialogue2graph.pipelines.core.dialogue import Dialogue
from dialogue2graph.metrics.llm_metrics import compare_graphs

In [5]:
from settings import EnvSettings

env_settings = EnvSettings()

In [7]:
import json

### Reading previously generated graph for "airport transfer" topic

In [10]:
with open("transfer.json") as f:
    data = json.load(f)

In [ ]:
Graph(data["graph"]).visualise_short("Transfer graph")

### Generating graph from dialogues with dia2graph generator where nodes are grouped via LLM

In [11]:
llm_generator = LLMGenerator()

In [14]:
dialogues = [Dialogue().from_list(seq["messages"]) for seq in data["dialogues"]]

In [ ]:
res_graph = llm_generator.invoke(dialogues)

In [ ]:
res_graph.visualise_short("Transfer graph generated from dialogues")

In [21]:
compare_graphs(res_graph, Graph(data["graph"]))

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-m3


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'value': True,
 'description': 'Nodes similarity: 0.9999998211860657, Nodes+edges similarity: 0.9464239478111267'}

### Generating graph from dialogues with dia2graph generator where nodes are grouped via algorithm

In [15]:
algo_generator = AlgoGenerator()

In [ ]:
res_graph = algo_generator.invoke(dialogues, model_name="gpt-4o")

In [ ]:
res_graph.visualise_short("Transfer graph generated from dialogues")

In [27]:
compare_graphs(res_graph, Graph(data["graph"]))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'value': True,
 'description': 'Nodes similarity: 0.9866758584976196, Nodes+edges similarity: 0.9464239478111267'}

### Demonstration of incremental approach where first graph generated from 1st dialogue, then next step is generating graph via LLM from previous graph and next dialogue

In [11]:
dialogues = [Dialogue().from_list(data["dialogues"][0]["messages"])]

In [ ]:
graph_1 = algo_generator.invoke(dialogues, model_name="gpt-4o")

In [ ]:
graph_1.visualise_short("Transfer from 1 dialogue")

In [20]:
dialogues = [Dialogue().from_list(data["dialogues"][1]["messages"])]

In [9]:
llm_extender = LLMExtender()

In [ ]:
graph_2, _ = llm_extender.invoke(dialogues, graph_1)

In [ ]:
graph_2.visualise_short("Transfer from 2 dialogue")

In [ ]:
dialogues = [Dialogue().from_list(data["dialogues"][2]["messages"])]
graph_3, _ = llm_extender.invoke(dialogues, graph_2)
graph_3.visualise_short("Transfer from 3 dialogue")

In [26]:
len(data["dialogues"])

45